In [1]:
# data science
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import datetime

# API
import requests
import json

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text, inspect, func
import plotly.express as px

In [4]:
# Create engine using the `Tesla.sqlite` database file
engine = create_engine("sqlite:///Tesla.sqlite")

In [5]:
# INSPECT to confirm existence

# Create the inspector and connect it to the engine
inspector_gadget = inspect(engine)

# Collect the names of tables within the database
tables = inspector_gadget.get_table_names()

# print metadata for each table
for table in tables:
    print(table)
    print("-----------")
    
    # get columns
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

supercharge_locations
-----------
id INTEGER
Supercharger TEXT
Street_Address TEXT
City TEXT
State TEXT
Country TEXT
Stalls INTEGER
kW REAL
Elev_m INTEGER
Open_Date DATE
Latitude REAL
Longitude REAL



In [6]:
query = "Select * from supercharge_locations"
df51 = pd.read_sql(text(query), con=engine)
df51.head(50)
df51.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5024 entries, 0 to 5023
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              5024 non-null   int64  
 1   Supercharger    5024 non-null   object 
 2   Street_Address  5024 non-null   object 
 3   City            5024 non-null   object 
 4   State           5024 non-null   object 
 5   Country         5024 non-null   object 
 6   Stalls          5024 non-null   int64  
 7   kW              5024 non-null   float64
 8   Elev_m          5024 non-null   int64  
 9   Open_Date       5024 non-null   object 
 10  Latitude        5024 non-null   float64
 11  Longitude       5024 non-null   float64
dtypes: float64(3), int64(3), object(6)
memory usage: 471.1+ KB


In [ ]:
df51.loc[df51.State == 'Limburg']

In [7]:
# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(autoload_with=engine)

# Print all of the classes mapped to the Base

print(list(Base.classes))

[<class 'sqlalchemy.ext.automap.supercharge_locations'>]


In [8]:
query = f"""
        SELECT Country, COUNT(*) AS Location_Count, AVG(Stalls) AS Average_Stalls
        FROM supercharge_locations
        GROUP BY  Country
        ORDER BY Location_Count DESC;
        """
df5 = pd.read_sql(text(query), con=engine)
df5.head(50)

,Country,Location_Count,Average_Stalls
0,USA,1791,10.933557
1,China,1701,6.283951
2,Canada,186,9.516129
3,Germany,170,14.023529
4,France,145,12.848276
5,South Korea,137,6.708029
6,United Kingdom,117,9.769231
7,Norway,106,15.358491
8,Taiwan,76,5.618421
9,Sweden,73,12.972603


In [ ]:
fig = px.scatter(df5, x="Location_Count", y="Average_Stalls",
	         size="Location_Count", color="Country",
                 hover_name="Country", log_x=True, size_max=60)
fig.show()

In [ ]:
query = f"""
        SELECT Country, State, City, Stalls, kW
        FROM supercharge_locations
        ORDER BY Country, State, Stalls, kW DESC;
        """
df6 = pd.read_sql(text(query), con=engine)
df6.head(50)

In [ ]:
fig = px.sunburst(df6, path=['Country', 'State', 'City'], values='Stalls',
                  color='Stalls', hover_data=['kW'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df6['Stalls'], weights=df6['kW']))
fig.show()

In [ ]:
query = f"""
        SELECT City, State, Country, COUNT(*) AS Location_Count
        FROM supercharge_locations
        GROUP BY City, State, Country
        ORDER BY Location_Count DESC;
        """
df1 = pd.read_sql(text(query), con=engine)
df1.head(50)

In [ ]:
query= f'''
    SELECT State, Country, AVG(Stalls) AS Average_Stalls, AVG(kW) AS Average_kW
    FROM supercharge_locations
    GROUP BY State
    ORDER BY Average_Stalls DESC, Average_kW DESC
    '''
df2 = pd.read_sql(text(query), con=engine)
display(df2.head(50))
df2.describe()

In [ ]:
query = f'''
        SELECT Country, COUNT(DISTINCT State) AS UniqueStatesCount
        FROM supercharge_locations
        GROUP BY Country
        ORDER BY UniqueStatesCount DESC;
        '''
df3 = pd.read_sql(text(query), con=engine)
df3.head(50)

In [ ]:
query = f'''

        SELECT DISTINCT State
        FROM supercharge_locations
        WHERE Country = 'USA';
        '''

df4 = pd.read_sql(text(query), con=engine)
df4.head(53)

In [12]:
query = f"""
        SELECT
            Country,
            State,
            City,
            SUM(Stalls) AS stalls,
            MIN(kW) AS min_kw,
            MAX(kW) AS max_kw
        FROM
            supercharge_locations
        ORDER BY
            stalls DESC;
        """

df9 = pd.read_sql(text(query), con=engine)
df9.head(53)

,Country,State,City,stalls,min_kw,max_kw
0,USA,FL,Daytona Beach,46052,72.0,250.0


In [16]:
query = f"""
SELECT Country, State, COUNT(*) AS Location_Count, SUM(Stalls) AS stalls, AVG(KW) AS Average_KW
        FROM supercharge_locations
        GROUP BY  Country, State
        ORDER BY Location_Count DESC;
         """
df8 = pd.read_sql(text(query), con=engine)
df8.head(53)

,Country,State,Location_Count,stalls,Average_KW
0,USA,CA,358,5692,195.153631
1,China,Guangdong,239,1613,200.000000
2,China,Zhejiang,219,1300,215.168950
3,China,Shanghai,173,1374,201.676301
4,USA,FL,134,1344,212.955224
5,China,Beijing,124,969,183.467742
6,China,Jiangsu,123,814,212.195122
7,USA,TX,123,1371,205.682927
8,United Kingdom,England,99,1001,202.525253
9,USA,NY,82,760,179.390244
